In [1]:

import pandas as pd
import numpy as np

In [2]:
## cleaning tool


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [3]:
##note KNN or other clusters might be helpful group the teams in smart way ... but not now.
#models


from sklearn.preprocessing import StandardScaler, MinMaxScaler 
std_scal = StandardScaler()
mm_scal = MinMaxScaler()


##no tuning 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


from sklearn.model_selection import train_test_split



##regression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

#classifiers (non-tree)
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDRegressor, SGDClassifier
from sklearn.svm import SVC


#tree-based classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor

##regression models
lr = Ridge(alpha=0.001) 
rfr = RandomForestRegressor(max_depth=3, random_state=0)
xgbr = XGBRegressor()

##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0)
svc = SVC()

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=3, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()



In [4]:
y_cols = ['Unnamed: 0', 'game_id', 'mp_date', 'season', 'home_team', 'away_team',
       'home_odds', 'away_odds', 'home_goals', 'away_goals', 'goal_diff_target', 'home_win',
       'settled_in', ]

x_cols = ['CF%', 'CSh%', 'CSv%', 'FF%', 'FSh%', 'FSv%', 'GDiff',
       'GF%', 'PDO', 'PENDiff', 'SF%', 'SDiff', 'Sh%', 'Sv']
columns_to_scale = ['CF%', 'CSh%', 'CSv%', 'FF%', 'FSh%', 'FSv%', 'GDiff',
       'GF%', 'PDO', 'PENDiff', 'SF%', 'SDiff', 'Sh%', 'Sv']  ##same

In [499]:
X = pd.read_csv('/Users/joejohns/data_bootcamp/GitHub/final_project_nhl_prediction/Data/Shaped_Data/data_LJ.csv')

X['season'] = X['season'].apply(int)
X['game_id'] = X['game_id'].apply(int)
X['mp_date'] = X['mp_date'].apply(int)
X['goal_diff_target'] = X['home_goals'] - X['away_goals']
X.loc[X['home_odds'] == X['away_odds'],  ['away_odds']]=110  #(!)
X.shape

(10385, 27)

In [500]:
def remove_early(X):
    filt_no_early  = (X['mp_date'].apply(lambda x : x% 10**4) < 900) | (1100 < X['mp_date'].apply(lambda x : x% 10**4))
    return X.loc[filt_no_early, : ].copy()  ## keep games < 900 and > 1100


In [501]:
##split data 

x = np.array(X.loc[(X['season'] <= 20152016), x_cols].copy())
  #features test-train
Y = X.loc[(X['season'] <= 20152016), y_cols].copy()
   #targets
y = np.array(Y['home_win']).reshape(-1,1)


              
x_16 = X.loc[(X['season'] == 20162017), x_cols].copy()  #features test-train
Y_16 = X.loc[(X['season'] == 20162017), y_cols].copy()   #targets
y_16 = np.array(Y_16['home_win']).reshape(-1,1)

x_17 = X.loc[(X['season'] > 20162017), x_cols].copy()  #features test-train
Y_17 = X.loc[(X['season'] > 20162017), y_cols].copy()   #targets
y_17 = np.array(Y_17['home_win']).reshape(-1,1)




In [8]:
X.loc[(X['season'] >= 20162017), :]['season'].value_counts()

20162017    1060
20172018    1052
Name: season, dtype: int64

In [9]:
X.loc[(X['season'] <= 20152016), :]['season'].value_counts()

20142015    1046
20112012    1037
20152016    1036
20132014     994
20082009     971
20102011     962
20092010     946
Name: season, dtype: int64

In [10]:
X.loc[(X['season'] <= 20152016), :]['season'].value_counts().sum()

6992

In [11]:
print(X.shape)
print(x.shape, Y.shape, y.shape)
print(x_16.shape, y_16.shape)
print(x_17.shape, y_17.shape)
print('sum ', y.shape[0], y_16.shape[0], y_17.shape[0], ' is: ', y.shape[0] + y_16.shape[0]+y_17.shape[0])

(9104, 27)
(6992, 14) (6992, 13) (6992, 1)
(1060, 14) (1060, 1)
(1052, 14) (1052, 1)
sum  6992 1060 1052  is:  9104


In [12]:
Y['season'].value_counts()

20142015    1046
20112012    1037
20152016    1036
20132014     994
20082009     971
20102011     962
20092010     946
Name: season, dtype: int64

In [13]:
Y_16['season'].value_counts()

20162017    1060
Name: season, dtype: int64

In [14]:
Y_17['season'].value_counts()

20172018    1052
Name: season, dtype: int64

In [502]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler 
std_scal = StandardScaler()
mm_scal = MinMaxScaler()


##no tuning 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)








##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0, max_iter = 1000)
svc = SVC(probability = True)

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=3, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()



##quick checks 
#for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
def fit_model(model, print_results = True, tuned = False)    
    model.fit(x_train_sc, y_train.ravel())
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    y_predt= model.predict(x_train_sc)  
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    acct = accuracy_score(y_train, y_predt)
    f1t = f1_score(y_train,y_predt)
    if tuned:
        tuned= 'WITH tuning'
    else
        tuned = 'NO tuning'
    
    if print_results:  
        print(str(model)[0:12], tuned+ 'TEST acc, f1: ', acc, f1, 'training acct, f1t: ', acct, f1t)
  
    


SyntaxError: invalid syntax (<ipython-input-502-0df0e7b0d549>, line 42)

In [30]:
x

array([[-1.07000000e+00,  3.88836773e+00, -1.50203584e+00, ...,
         0.00000000e+00,  6.54126895e+00, -2.47000000e+00],
       [-5.33000000e+00,  3.22158462e-01,  7.96868447e-01, ...,
        -8.69000000e+00,  7.75485350e-01,  2.28000000e+00],
       [-1.13900000e+01,  1.29385506e+00, -7.31934901e-01, ...,
        -1.05100000e+01,  2.16503340e+00, -2.47000000e+00],
       ...,
       [-1.07000000e+00, -1.70848125e-03, -6.54668512e-02, ...,
        -2.04000000e+00, -3.28207435e-01,  5.00000000e-01],
       [ 4.45000000e+00, -3.02955417e-01,  7.35640597e-01, ...,
         4.42000000e+00, -2.41020059e-01,  1.24000000e+00],
       [ 3.87000000e+00,  2.48248880e-01,  4.01685000e-01, ...,
         6.00000000e+00,  3.36489035e-01,  3.90000000e-01]])

In [124]:
#check on 2016

x_test2 = std_scal.transform(x_16).copy()
y_test2 = y_16.copy()


##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], '20162017, no tuning, TEST: ', acc, f1) #, 'training : ', acct, f1t)

SVC() 20162017, no tuning, TEST:  0.5773584905660377 0.6985195154777928
RidgeClassif 20162017, no tuning, TEST:  0.5311320754716982 0.6452533904354033
RandomForest 20162017, no tuning, TEST:  0.5783018867924529 0.7091737150292778
GaussianNB() 20162017, no tuning, TEST:  0.5735849056603773 0.627677100494234
LogisticRegr 20162017, no tuning, TEST:  0.5528301886792453 0.6814516129032259
BaggingClass 20162017, no tuning, TEST:  0.5132075471698113 0.5401069518716578
GradientBoos 20162017, no tuning, TEST:  0.5707547169811321 0.6838082001389855
XGBClassifie 20162017, no tuning, TEST:  0.5311320754716982 0.6120218579234973


In [123]:
#check on 2017

x_test2 = std_scal.transform(x_17).copy()
y_test2 = y_17.copy()


##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], '20172018, no tuning, TEST: ', acc, f1) #, 'training : ', acct, f1t)

SVC() 20172018, no tuning, TEST:  0.5884030418250951 0.6931254429482636
RidgeClassif 20172018, no tuning, TEST:  0.5884030418250951 0.6913756236635781
RandomForest 20172018, no tuning, TEST:  0.6036121673003803 0.7098121085594988
GaussianNB() 20172018, no tuning, TEST:  0.5731939163498099 0.6286186931348222
LogisticRegr 20172018, no tuning, TEST:  0.5874524714828897 0.6917613636363635
BaggingClass 20172018, no tuning, TEST:  0.529467680608365 0.5638766519823788
GradientBoos 20172018, no tuning, TEST:  0.5998098859315589 0.6906686260102866
XGBClassifie 20172018, no tuning, TEST:  0.55893536121673 0.6340694006309148


#can do version where we get y regr version at beinning ... do all teh regression stuff ... 
then for later cells do y = v_win(y) for classifiers ... 

##regression models now ...

y = Y['goal_diff_target'].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)


lr = Ridge(alpha=50000) 
rfr = RandomForestRegressor(max_depth=4, random_state=0)
xgbr = XGBRegressor()


##quick checks 
for model in [lr, rfr, xgbr]:
    model.fit(x_train_sc, y_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    print(y_pred[0:5])
    y_predw = v_win(y_pred)
    y_predt= model.predict(x_train_sc)  
    y_predwt = v_win(y_predt) 
    y_trainw = v_win(y_train)
    y_testw = v_win(y_test)  #same as usual win/loss
    acc = accuracy_score(y_testw, y_predw)
    f1 = f1_score(y_testw,y_predw)
    acct = accuracy_score(y_trainw, y_predwt)
    f1t = f1_score(y_trainw,y_predwt)
    
  
    print(str(model)[0:20], 'TEST: ', acc, f1 ,'training : ', acct, f1t)
    

In [122]:
## some tries at tuning here 


from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12, stratify=y)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)


##classifier models
lrc2 = RidgeClassifier(alpha =0.2)
gnb2 = GaussianNB()
lgr2 = LogisticRegression(random_state = 0, C =10**4, max_iter = 500)
svc2 = SVC(kernel = 'rbf', C =10, probability = True)  #why would you ever have it false? 

#tree-based classifiers
rfc2 =  RandomForestClassifier(max_depth=10, random_state=0, n_estimators = 40)
bc2 = BaggingClassifier(n_estimators  = 60, max_samples = 0.85)
gbc2 = GradientBoostingClassifier(learning_rate =0.1, n_estimators = 40, max_depth =4 )
xgbc2 = XGBClassifier(n_estimators= 45, eta=0.05)

##quick checks 
for model in [svc2, lrc2, rfc2, gnb2, lgr2, bc2, gbc2, xgbc2]:
    model.fit(x_train_sc, y_train.ravel())
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    y_predt= model.predict(x_train_sc)  
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    acct = accuracy_score(y_train, y_predt)
    f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], 'with tuning TEST: ', acc, f1, 'training : ', acct, f1t)

SVC(C=10, pr with tuning TEST:  0.5668334524660472 0.6666666666666667 training :  0.6264974074736277 0.7127732710023373
RidgeClassif with tuning TEST:  0.5675482487491065 0.6606842400448683 training :  0.5758984444841766 0.6692693809258226
RandomForest with tuning TEST:  0.5668334524660472 0.6521239954075775 training :  0.8119077418201323 0.8471818710052295
GaussianNB() with tuning TEST:  0.5689778413152251 0.6045901639344262 training :  0.5649919542284999 0.606119475473531
LogisticRegr with tuning TEST:  0.5639742673338098 0.6545866364665911 training :  0.5730377257285894 0.6658270361041141


/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


BaggingClass with tuning TEST:  0.5503931379556827 0.6162294081757168 training :  1.0 1.0
GradientBoos with tuning TEST:  0.5725518227305219 0.6621468926553672 training :  0.6676202395851958 0.7381321312860967
[15:41:06] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifie with tuning TEST:  0.5539671193709793 0.6450511945392492 training :  0.7201859467191132 0.7769066286528866


In [118]:
#check on 2016

x_test2 = std_scal.transform(x_16).copy()
y_test2 = y_16.copy()


##quick checks 
#for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
for model in [svc2, lrc2, rfc2, gnb2, lgr2, bc2, gbc2, xgbc2]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], '20162017, tuned, TEST: ', acc, f1) #, 'training : ', acct, f1t)

SVC(C=10, pr 20162017, tuned, TEST:  0.539622641509434 0.6291793313069909
RidgeClassif 20162017, tuned, TEST:  0.4839622641509434 0.5939123979213066
RandomForest 20162017, tuned, TEST:  0.5660377358490566 0.6904441453566622
GaussianNB() 20162017, tuned, TEST:  0.5707547169811321 0.6291768541157293
LogisticRegr 20162017, tuned, TEST:  0.44716981132075473 0.4904347826086956
BaggingClass 20162017, tuned, TEST:  0.5367924528301887 0.6316579144786196
GradientBoos 20162017, tuned, TEST:  0.5622641509433962 0.672316384180791
XGBClassifie 20162017, tuned, TEST:  0.5716981132075472 0.6811797752808989


In [119]:
#check on 2017

x_test2 = std_scal.transform(x_17).copy()
y_test2 = y_17.copy()


##quick checks 
#for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
for model in [svc2, lrc2, rfc2, gnb2, lgr2, bc2, gbc2, xgbc2]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12],  '20172018, tuned, TEST: ', acc, f1) #, 'training : ', acct, f1t)

SVC(C=10, pr 20172018, tuned, TEST:  0.5836501901140685 0.6857962697274033
RidgeClassif 20172018, tuned, TEST:  0.5960076045627376 0.6975088967971531
RandomForest 20172018, tuned, TEST:  0.5931558935361216 0.6857562408223201
GaussianNB() 20172018, tuned, TEST:  0.5674904942965779 0.6211490424646129
LogisticRegr 20172018, tuned, TEST:  0.5960076045627376 0.6992215145081387
BaggingClass 20172018, tuned, TEST:  0.5484790874524715 0.6141348497156783
GradientBoos 20172018, tuned, TEST:  0.5950570342205324 0.6921965317919075
XGBClassifie 20172018, tuned, TEST:  0.5988593155893536 0.6901615271659325


In [22]:
#Let's go with lgr, lgr2 (tuned or not) for betting investigation

In [23]:
x_16 = std_scal.transform(x_16).copy()


In [26]:
svc2.predict_proba(x_16)

array([[0.52407527, 0.47592473],
       [0.58086487, 0.41913513],
       [0.41006043, 0.58993957],
       ...,
       [0.37652706, 0.62347294],
       [0.44296909, 0.55703091],
       [0.3999811 , 0.6000189 ]])

In [28]:
gnb.predict_proba(x_16)[0:, 0]

array([0.00380656, 0.32217664, 0.21205196, ..., 0.07010458, 0.54824593,
       0.05309209])

In [29]:
y_16

array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]])

In [ ]:
##make betting strategy on x_16 test it on x_17  ... and other seasons if needed

In [38]:
def implied_proba(odds):
    if odds > 0: 
        return 100/(odds+100)    #bet 100 to get 100+odds; profit = odds
    if odds < 0:
        return (-odds)/(-odds + 100)   #bet |odds| to get 100+|odds|; profit = 100
    
v_impl_proba = np.vectorize(implied_proba)

In [412]:
dic = {}  #x_16, lgr 0.55 ... Request rfc 0.58 x_16 and 0.6 on x_17 (later)

dic['actual'] = [ x[0] for x in list(y_17)]
dic['lgr_pred'] = list(rfc.predict(x_17))
dic['lgr_conf_1'] = [round(x,4) for x in rfc.predict_proba(x_17)[0:, 1]]


dic['lgr_conf_0'] =[round(x,4) for x in rfc.predict_proba(x_17)[0:, 0]]


dic['away_odds'] = list(Y_17['away_odds'])
dic['home_odds'] = list(Y_17['home_odds'])


df_lgr = pd.DataFrame(dic)

In [453]:
dic = {}  #x_16, lgr 0.55 ... Request rfc 0.58 x_16 and 0.6 on x_17 (later)

dic['actual'] = [ x[0] for x in list(y_16)]
dic['lgr_pred'] = list(rfc.predict(x_16))
dic['lgr_conf_1'] = [round(x,4) for x in rfc.predict_proba(x_16)[0:, 1]]


dic['lgr_conf_0'] =[round(x,4) for x in rfc.predict_proba(x_16)[0:, 0]]


dic['away_odds'] = list(Y_16['away_odds'])
dic['home_odds'] = list(Y_16['home_odds'])


df_lgr = pd.DataFrame(dic)

In [454]:
df_lgr.columns

Index(['actual', 'lgr_pred', 'lgr_conf_1', 'lgr_conf_0', 'away_odds',
       'home_odds'],
      dtype='object')

In [455]:
df_lgr['away_impl_proba'] = v_impl_proba(df_lgr['away_odds'])

df_lgr['home_impl_proba'] = v_impl_proba(df_lgr['home_odds'])


df_lgr['conf_1_sub_home_impl'] = df_lgr['lgr_conf_1'] - df_lgr['home_impl_proba']
df_lgr['conf_0_sub_away_impl'] = df_lgr['lgr_conf_0'] - df_lgr['away_impl_proba']

In [456]:
df_lgr = df_lgr.loc[:, ['actual', 'lgr_pred', 'lgr_conf_1', 'home_impl_proba',        'conf_1_sub_home_impl', 'conf_0_sub_away_impl',    'lgr_conf_0', 'away_impl_proba', 
'home_odds',  'away_odds']].copy()

In [480]:
df['away_odds']


31      130.0
54      165.0
127    -160.0
134     150.0
358     118.0
489     180.0
545     130.0
774    -165.0
794     225.0
818     215.0
976    -265.0
990    -235.0
993    -275.0
1017   -275.0
1041    130.0
Name: away_odds, dtype: float64

In [458]:
df_100.describe()

,actual,lgr_pred,lgr_conf_1,home_impl_proba,conf_1_sub_home_impl,conf_0_sub_away_impl,lgr_conf_0,away_impl_proba,home_odds,away_odds,model_conf,bet_HA,bet,potential_ROI,pay_out,profit,cumul_profit,total_profit,total_bet,total_ROI
count,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.0,832.000000,832.000000,832.000000,832.000000,8.320000e+02,832.0,8.320000e+02
mean,0.578125,0.674279,0.545259,0.570073,-0.024814,-0.012757,0.454741,0.467498,-100.808894,51.542067,0.090517,0.674279,100.0,1.772520,101.777332,1.777332,850.240036,1.478740e+03,83200.0,1.017773e+00
std,0.494156,0.468926,0.092849,0.080177,0.081623,0.081914,0.092849,0.082110,116.945988,131.413516,0.185697,0.468926,0.0,0.259088,89.319910,89.319910,913.395565,7.962865e-12,0.0,8.664948e-15
min,0.000000,0.000000,0.377000,0.338983,-0.245457,-0.255537,0.284400,0.250000,-370.000000,-235.000000,-0.246000,0.000000,100.0,1.270270,0.000000,-100.000000,-936.750000,1.478740e+03,83200.0,1.017773e+00
25%,0.000000,0.000000,0.473475,0.527743,-0.089163,-0.074996,0.382275,0.400000,-170.000000,-108.250000,-0.053050,0.000000,100.0,1.588235,0.000000,-100.000000,74.272500,1.478740e+03,83200.0,1.017773e+00
50%,1.000000,1.000000,0.542450,0.579832,-0.037332,0.002752,0.457550,0.458716,-138.000000,118.000000,0.084900,1.000000,100.0,1.714286,150.000000,50.000000,976.345000,1.478740e+03,83200.0,1.017773e+00
75%,1.000000,1.000000,0.617725,0.629630,0.038971,0.048214,0.526525,0.519806,-111.750000,150.000000,0.235450,1.000000,100.0,1.892857,172.460000,72.460000,1438.437500,1.478740e+03,83200.0,1.017773e+00
max,1.000000,1.000000,0.715600,0.787234,0.219256,0.217985,0.623000,0.701493,195.000000,300.000000,0.431200,1.000000,100.0,2.850000,285.000000,185.000000,3376.510000,1.478740e+03,83200.0,1.017773e+00


In [491]:



def find_pay_off(conf_1_at_least = 0 ,  conf_0_at_least =0 , diff_home_C_1_at_least =0, diff_away_C_0_at_least =0, type_bet = "get_100"  ): #or "bet_100"
    #
    ##return df with columns for each season in question:
    #type_bet,  total_invested, total_earned, profit, ROI, 
    ##rows should be: avg for season in question let's do x_16 for now
    dic = {}
    dic['bet_strategy'] = type_bet
    filter_thresh = ( df_lgr['lgr_conf_1'] >=  conf_1_at_least)| (df_lgr['lgr_conf_0'] >= conf_0_at_least)
    filter_diff_thresh = (df_lgr['conf_1_sub_home_impl']  >= diff_home_C_1_at_least) | (df_lgr['conf_0_sub_away_impl']  >= diff_away_C_0_at_least)
   
    df = df_lgr[filter_thresh &  filter_diff_thresh].copy()
    if type_bet == "bet_100":
        pass
    #assume type = "get_100"
    #pred = df['lgr_pred'].copy
    df['ones'] =1
    df['non_pred'] = df['ones'] - df['lgr_pred']
    df['model_conf'] = df_lgr['lgr_conf_1'] - df_lgr['lgr_conf_0']
    df['bet_HA'] = df['lgr_pred']
    df['bet'] = df['lgr_pred']*df['home_impl_proba'] + df['non_pred']*df['away_impl_proba']
    df['bet'] = 100*df['bet']
    df['bet'] = df['bet'].apply(lambda x: round(x,2))
    df['potential_ROI'] = df['lgr_pred']*(1/df['home_impl_proba']) + df['non_pred']*(1/df['away_impl_proba'])
    df['pay_out'] = df['actual']*100
    df['profit'] = df['pay_out'] - df['bet']       
    df['cumul_profit'] = 0
    
    for i in df.index:
        df.loc[i, 'cumul_profit'] = df.loc[:i, 'profit'].sum()
    df['total_profit'] = df['profit'].sum()    
    df['total_bet'] = df['bet'].sum()    
    df['total_ROI'] = df['total_profit']/df['total_bet']    
    
    df.drop(columns = ['ones', 'non_pred'], inplace = True)
    return df

def find_pay_off_bet_100(conf_1_at_least = 0 ,  conf_0_at_least =0 ,conf_thresh=0 , diff_home_C_1_at_least =0, diff_away_C_0_at_least =0, type_bet = "Bet_100"):  #or "bet_100"
    #
    ##return df with columns for each season in question:
    #type_bet,  total_invested, total_earned, profit, ROI, 
    ##rows should be: avg for season in question let's do x_16 for now
    dic = {}
    dic['bet_strategy'] = type_bet
    filter_thresh = (( df_lgr['lgr_conf_1'] >=  conf_1_at_least) + (df_lgr['lgr_conf_0'] >= conf_0_at_least)).copy()
    filter_diff_thresh = ((df_lgr['conf_1_sub_home_impl']  >= diff_home_C_1_at_least) + (df_lgr['conf_0_sub_away_impl']  >= diff_away_C_0_at_least)).copy()
    filter_conf_thresh = ((df_lgr['lgr_conf_1'] - df_lgr['lgr_conf_0']) > conf_thresh ).copy() 
   
    df = df_lgr[ filter_diff_thresh & filter_conf_thresh].copy() 
    #df = df_lgr[ filter_diff_thresh & filter_thresh].copy() 
    
    if type_bet == "bet_100":
        pass
    #assume type = "get_100"
    #pred = df['lgr_pred'].copy
    df['ones'] =1
    df['non_pred'] = df['ones'] - df['lgr_pred']
    df['model_conf'] = df_lgr['lgr_conf_1'] - df_lgr['lgr_conf_0']
    df['bet_HA'] = df['lgr_pred']
    
    df['bet'] = 100
    df['potential_ROI'] = df['lgr_pred']*(1/df['home_impl_proba']) + df['non_pred']*(1/df['away_impl_proba'])
    df['pay_out'] = df['actual']*df['potential_ROI']*100
    df['pay_out'] = df['pay_out'].apply(lambda x: round(x,2))
    df['profit'] = df['pay_out'] - df['bet']                     
    df['cumul_profit'] = 0
    
    for i in df.index:
        df.loc[i, 'cumul_profit'] = df.loc[:i, 'profit'].sum()
    df['total_profit'] = df['profit'].sum()    
    df['total_bet'] = df['bet'].sum()    
    df['total_ROI'] = (df['total_profit']+df['total_bet'])/df['total_bet']    
    
    df.drop(columns = ['ones', 'non_pred'], inplace = True)
    return df
        
    
    

In [498]:
df_lgr.loc[df_lgr.loc[df_lgr['home_odds'] == df_lgr['away_odds'], :]

,actual,lgr_pred,lgr_conf_1,home_impl_proba,conf_1_sub_home_impl,conf_0_sub_away_impl,lgr_conf_0,away_impl_proba,home_odds,away_odds
2,0.0,1.0,0.6159,0.52381,0.09209,-0.13971,0.3841,0.52381,-110.0,-110.0
97,1.0,0.0,0.4192,0.52381,-0.10461,0.05699,0.5808,0.52381,-110.0,-110.0
115,1.0,1.0,0.5492,0.52381,0.02539,-0.07301,0.4508,0.52381,-110.0,-110.0
611,0.0,0.0,0.4605,0.52381,-0.06331,0.01569,0.5395,0.52381,-110.0,-110.0
688,1.0,0.0,0.4543,0.52381,-0.06951,0.02189,0.5457,0.52381,-110.0,-110.0
765,1.0,0.0,0.4914,0.52381,-0.03241,-0.01521,0.5086,0.52381,-110.0,-110.0


In [493]:
np.abs(-6)

6

In [ ]:
def v_fav(x):
    if x <0:
        return 1

In [ ]:

##bet the spread mofo!

def find_pay_off_spread(conf_1_at_least = 0 ,  conf_0_at_least =0 , diff_home_C_1_at_least =0, diff_away_C_0_at_least =0, type_bet = "bet the spread!"  ): #or "bet_100"
    #
    ##return df with columns for each season in question:
    #type_bet,  total_invested, total_earned, profit, ROI, 
    ##rows should be: avg for season in question let's do x_16 for now
    dic = {}
    dic['bet_strategy'] = type_bet
    filter_thresh = ( df_lgr['lgr_conf_1'] >=  conf_1_at_least)| (df_lgr['lgr_conf_0'] >= conf_0_at_least)
    filter_diff_thresh = (df_lgr['conf_1_sub_home_impl']  >= diff_home_C_1_at_least) | (df_lgr['conf_0_sub_away_impl']  >= diff_away_C_0_at_least)
   
    #df = df_lgr[filter_thresh &  filter_diff_thresh].copy()
    df = df_lgr.copy()
    if type_bet == "bet_100":
        pass
    #assume type = "get_100"
    #pred = df['lgr_pred'].copy

    df['bet_HA'] = df['lgr_pred']
    ##fir
    
    
    df['bet'] = (1-df['lgr_pred'])*np.abs(df['away_odds']) + df['lgr_pred']*np.abs(df['home_odds'])
    
 
   
    df['pay_out'] = (1-df['actual'])*(1-df['lgr_predict'])*np.abs(df['away_odds'])   +  df['actual']*df['lgr_predict']*df['lgr_pred']*np.abs(df['home_odds'])  
    
    df['profit'] = df['pay_out'] - df['bet']       
    df['cumul_profit'] = 0
    
    for i in df.index:
        df.loc[i, 'cumul_profit'] = df.loc[:i, 'profit'].sum()
    df['total_profit'] = df['profit'].sum()    
    df['total_bet'] = df['bet'].sum()    
    df['total_ROI'] = df['total_profit']/df['total_bet']    
    
    df.drop(columns = ['ones', 'non_pred'], inplace = True)
    return df

    
    

In [460]:
df['profit'].sum()

247.64000000000016

In [461]:
cols = ['actual', 'lgr_pred', 'lgr_conf_1', 'home_impl_proba',
       'conf_1_sub_home_impl', 'conf_0_sub_away_impl', 
       'away_impl_proba',  'bet_HA',
       'bet', 'pay_out', 'profit', 'total_profit','cumul_profit', 'total_bet', 'total_ROI' ]

In [462]:
df_100.shape

(832, 20)

In [463]:
df_100.loc[800:900,cols]

,actual,lgr_pred,lgr_conf_1,home_impl_proba,conf_1_sub_home_impl,conf_0_sub_away_impl,away_impl_proba,bet_HA,bet,pay_out,profit,total_profit,cumul_profit,total_bet,total_ROI
800,1.0,0.0,0.4693,0.392157,0.077143,-0.105664,0.636364,0.0,100,157.14,57.14,1478.74,1243.16,83200,1.017773
801,1.0,1.0,0.5885,0.649123,-0.060623,0.034142,0.377358,1.0,100,154.05,54.05,1478.74,1297.21,83200,1.017773
802,0.0,1.0,0.7106,0.696970,0.013630,-0.055428,0.344828,1.0,100,0.00,-100.00,1478.74,1197.21,83200,1.017773
803,1.0,1.0,0.6690,0.607843,0.061157,-0.094532,0.425532,1.0,100,164.52,64.52,1478.74,1261.73,83200,1.017773
804,1.0,1.0,0.5809,0.539171,0.041729,-0.088289,0.507389,1.0,100,185.47,85.47,1478.74,1347.20,83200,1.017773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,1.0,1.0,0.5981,0.696970,-0.098870,0.057072,0.344828,1.0,100,143.48,43.48,1478.74,984.31,83200,1.017773
897,1.0,1.0,0.6216,0.534884,0.086716,-0.133795,0.512195,1.0,100,186.96,86.96,1478.74,1071.27,83200,1.017773
898,0.0,0.0,0.4866,0.591837,-0.105237,0.068956,0.444444,0.0,100,0.00,-100.00,1478.74,971.27,83200,1.017773
899,1.0,1.0,0.5166,0.619772,-0.103172,0.071877,0.411523,1.0,100,161.35,61.35,1478.74,1032.62,83200,1.017773


In [464]:
df_100 = find_pay_off_bet_100(conf_1_at_least = 0 ,  conf_0_at_least =0 ,conf_thresh=0 , diff_home_C_1_at_least =0, diff_away_C_0_at_least =0, type_bet = "Bet_100"):  #or "bet_100"
    #

SyntaxError: invalid syntax (<ipython-input-464-b49c3ed7f79e>, line 1)

In [476]:
df_100 = find_pay_off_bet_100(conf_1_at_least = 0, conf_0_at_least = 0, conf_thresh = -100,diff_home_C_1_at_least =0.2, diff_away_C_0_at_least =0.1, type_bet = "bet_100"  )
df_100['total_profit']

31     -262.9
54     -262.9
134    -262.9
358    -262.9
489    -262.9
545    -262.9
774    -262.9
794    -262.9
818    -262.9
1041   -262.9
Name: total_profit, dtype: float64

In [490]:
df = find_pay_off(conf_1_at_least = 0.2,  conf_0_at_least = 0.2, diff_home_C_1_at_least = -10, diff_away_C_0_at_least = -10, type_bet = "get_100"  )
print(accuracy_score(df['actual'], df['lgr_pred'], ))
df['total_profit']                   #OR#                  #AND#                      ##OR##

0.589622641509434


0      -5111.77
1      -5111.77
2      -5111.77
3      -5111.77
4      -5111.77
         ...   
1055   -5111.77
1056   -5111.77
1057   -5111.77
1058   -5111.77
1059   -5111.77
Name: total_profit, Length: 1060, dtype: float64

In [467]:
df_lgr.describe()

,actual,lgr_pred,lgr_conf_1,home_impl_proba,conf_1_sub_home_impl,conf_0_sub_away_impl,lgr_conf_0,away_impl_proba,home_odds,away_odds
count,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.00000
mean,0.546226,0.745283,0.548657,0.572060,-0.023402,-0.014774,0.451343,0.466117,-101.160377,51.90566
std,0.498094,0.435908,0.071981,0.085215,0.047763,0.048438,0.071981,0.086603,122.425390,134.58566
min,0.000000,0.000000,0.395000,0.307692,-0.245919,-0.237642,0.302100,0.270270,-340.000000,-275.00000
25%,0.000000,0.000000,0.499175,0.521531,-0.055223,-0.044192,0.400075,0.400000,-170.000000,-111.00000
50%,1.000000,1.000000,0.549800,0.574468,-0.023808,-0.014130,0.450200,0.465116,-135.000000,115.00000
75%,1.000000,1.000000,0.599925,0.629630,0.004058,0.015623,0.500825,0.526066,-109.000000,150.00000
max,1.000000,1.000000,0.697900,0.772727,0.206837,0.211357,0.605000,0.733333,225.000000,270.00000


In [482]:
accuracy_score(df_lgr['actual'], df_lgr['lgr_pred'], )

0.589622641509434

In [469]:
print(df.loc[(df['bet_HA'] == 1)&(df['pay_out'] >0), ['profit']].sum(),
df.loc[(df['bet_HA'] == 1)&(df['pay_out'] >0), ['profit']].count(),
'avg loss ', df.loc[(df['bet_HA'] == 1)&(df['pay_out'] >0), ['profit']].sum()/df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].count()     )

print(df.loc[(df['bet_HA'] == 1)&(df['pay_out'] ==0), ['profit']].sum(),
df.loc[(df['bet_HA'] == 1)&(df['pay_out'] ==0), ['profit']].count(),
'avg loss ', df.loc[(df['bet_HA'] == 1)&(df['pay_out'] ==0), ['profit']].sum()/df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].count()     )

print(df.loc[(df['bet_HA'] == 0)&(df['pay_out'] >0), ['profit']].sum(),
df.loc[(df['bet_HA'] == 0)&(df['pay_out'] >0), ['profit']].count(),
'avg loss ', df.loc[(df['bet_HA'] == 0)&(df['pay_out'] >0), ['profit']].sum()/df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].count()     )

print(df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].sum(),
df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].count(),
'avg loss ', df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].sum()/df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].count()     )


print(df['profit'].sum())
print('avg loss', df['profit'].sum()/df['profit'].count())


profit    11874.04
dtype: float64 profit    308
dtype: int64 avg loss  profit    108.936147
dtype: float64
profit   -12862.43
dtype: float64 profit    216
dtype: int64 avg loss  profit   -118.003945
dtype: float64
profit    3268.54
dtype: float64 profit    75
dtype: int64 avg loss  profit    29.986606
dtype: float64
profit   -6181.82
dtype: float64 profit    109
dtype: int64 avg loss  profit   -56.713945
dtype: float64
-3901.6700000000005
avg loss -5.510833333333334


In [470]:
(df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']] <0).sum()

profit    109
dtype: int64

In [471]:


dic2['lgr_pred'] = list(lgr.predict(x_16))
dic2['gnb_pred'] = list(gnb.predict(x_16))
dic2'svc_proba'] = list(svc.predict_proba(x_16)[:,1])


SyntaxError: invalid syntax (<ipython-input-471-6cde86c145d1>, line 3)

In [472]:
predictions = gnb.predict(x_17)
actual = y_17
confusionMatrix = confusion_matrix(actual, predictions)
display(confusionMatrix)

tn = confusionMatrix[0][0]
fp = confusionMatrix[0][1]
fn = confusionMatrix[1][0]
tp = confusionMatrix[1][1]
actualYes = fn + tp 
actualNo = tn + fp
predictedYes = fp + tp
predictedNo = tn + fn

print('When home team wins, classifier predicts they will win %6.2f%% of the time' % (tp / actualYes * 100))
print('When home team loses, classifier predicts they will win %6.2f%% of the time' % (fp / actualNo * 100))
print('When home team loses, classifier predicts they will lose %6.2f%% of the time' % (tn / actualNo * 100))
print('When classifer predicts home team will win, home team actually wins %6.2f%% of the time' % (tp / predictedYes * 100))
print('When classifer predicts home team will lose, home team actually loses %6.2f%% of the time' % (tn / predictedNo * 100))

array([[379,  70],
       [500, 103]])

When home team wins, classifier predicts they will win  17.08% of the time
When home team loses, classifier predicts they will win  15.59% of the time
When home team loses, classifier predicts they will lose  84.41% of the time
When classifer predicts home team will win, home team actually wins  59.54% of the time
When classifer predicts home team will lose, home team actually loses  43.12% of the time


In [473]:
## is the betting and calcs being done correctly?  62% accuracy should easily have a profit ... 


##try all models

##try regression 

##think about this "confidence diff " Leung mentioned 



##also ... sigh ... put in the FW% thing I guess ... [that was what I was struggling with calculating yesterday ]
